# 1. Example 1

### 1.1. Base Class: Person
- The `Person` base class has a private attribute `_name` and public attribute `name`.
- The `name` attribute is managed using a property, which consists of a getter, setter, and deleter.
- The getter(`@property`) returns the value of the private attribute `_name`.
- The setter(`@property.setter`) ensures that the assigned value is a string, raising a `TypeError` otherwise.
- The deleter(`@property.deleter`) raises an `AttributeError` to prevent attribute deletion. 

In [1]:
class Person:
    def __init__(self, name: str) -> None:
        self.name = name

    # Getter function
    @property
    def name(self) -> str:
        return self._name

    # Setter function
    @name.setter
    def name(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError(f'Expected a string, but got type {type(value).__name__.upper()}')
        self._name = value

    @name.deleter
    def name(self) -> None:
        raise AttributeError("Cannot delete attribute")

### 1.2. Sub Class: SubPerson

- The `SubPerson` class is a subclass of `Person`.
- It overrides the getter, setter, and deleter methods for the `name` property.
- The getter prints a message and then calls the getter of the superclass using `super().name`.
- The setter prints a message, then calls the setter of the superclass using `super(SubPerson, SubPerson).name.__set__(self, value)`

In [2]:
class SubPerson(Person):
    @property
    def name(self) -> str:
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value: str) -> None:
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self) -> None:
        print('Deleting a name')
        super(SubPerson, SubPerson).name.__delete__(self)

In [3]:
a = Person('Benjamin')
a.name

'Benjamin'

In [4]:
a.name = 'Dave'
a.name

'Dave'

In [5]:
try:
    a.name = 42
except TypeError as e:
    print(e)

Expected a string, but got type INT


### 1.3. Conclusion 
This demonstrates how property methods can be customized in subclasses while maintaining the property management logic from the base class. 

# 2.Deeper Look

### 2.1. Breaking down: `super(SubPerson, SubPerson).name.__set__(slef, value)`

1. **`super(SubPerson, SubPerson)`**: 
   - `super()` is a built-in function in Python that returns a temporary object of the superclass, allowing you to call its methods.
   - `super(SubPerson, SubPerson)` specifies that we want to operate on the superclass of `SubPerson` within the context of the `SubPerson` class.

2. **`.name`**: 
   - This accesses the `name` property of the superclass.

3. **`.__set__(self, value)`**: 
   - This calls the `__set__` method of the property, which is part of the descriptor protocol in Python.
   - The `__set__` method is called when the property is assigned a new value.
   - The method takes two arguments: `self` (the instance of the class) and `value` (the value being assigned).

### 2.2. Putting it all together, the line is essentially saying:

- Use the `super()` function to get the superclass of `SubPerson` within the context of the `SubPerson` class.
- Access the `name` property of that superclass.
- Call the `__set__` method of the property, passing `self` (the instance of `SubPerson`) and `value` (the new value being assigned to `name`).

### 2.3. Why is this used?

- In this specific context, the code is intercepting the setting of the `name` property to add custom behavior before or after the actual setting.
- The `print('Setting name to', value)` statement just before this line indicates that a message is printed before the actual setting, showing that custom behavior is added.
- By using `super()`, the code ensures that it is calling the `__set__` method of the superclass, allowing any additional logic implemented in the superclass to execute.

# 3. Example 2

### 3.1. Scenario

- Let's consider a scenario where you want to enforce additional constraints on a property, such as limiting the length of a person's name in the `Person` class. The `SubPerson` subclass can then customize this behavior in the setter. 

In [6]:
class Person:
    def __init__(self, name: str) -> None:
        self.name = name

    @property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, value) -> None:
        if not isinstance(value, str):
            raise TypeError(f'Expected a string but got {type(value).__name__.upper()}')
        if len(value) > 20:
            raise ValueError('Name is too long')
        self._name = value

In [7]:
class SubPerson(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting the name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

In [8]:
subperson = SubPerson('Jane Doe')
print(subperson.name)

Setting the name to Jane Doe
Jane Doe


### 3.2. Conclusion

- The custom behavior in the `SubPerson` class is triggered, printing a message before calling the setter of the superclass. 
- This example demonstrates how the `SubPerson` class can customize the behavior of the `name` property, adding custom actions before or after the actual setting logic implemented in the superclass. 

In [9]:
try:
    subperson.name = 'Jane Doe Smith Johnson'
    print(subperson.name)
except ValueError as e:
    print(e)

Setting the name to Jane Doe Smith Johnson
Name is too long
